In [ ]:
# Censo Nacional

In [ ]:
# V.3.0.1 Basdo en voila

import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML
import voila
import markdown

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        # Agregar la columna "Producto" con el nombre del archivo sin extensión
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns
print("Campos disponibles:")
print(fields)

# Obtener la lista de productos
product_list = data["Producto"].unique()

# Obtener la lista de municipios
municipio_list = data["Municipio"].unique()

# Crear las listas desplegables para seleccionar las variables y el tipo de gráfico
variable_dropdown = widgets.Dropdown(options=["Area Sembrada", "Area Cosechada", "Produccion (ton)", "Rendimiento (ha/ton)"], description="Variable:")
product_dropdown = widgets.SelectMultiple(options=product_list, description="Productos:")
municipio_dropdown = widgets.SelectMultiple(options=municipio_list, description="Municipios:")
index_dropdown = widgets.Dropdown(options=fields, description="Index:")
x_axis_dropdown = widgets.Dropdown(options=fields, description="Eje X:")
y_axis_dropdown = widgets.Dropdown(options=fields, description="Eje Y:")
chart_type_dropdown = widgets.Dropdown(options=["bar", "line", "scatter"], description="Tipo de gráfico:")

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Función para generar y mostrar la tabla y el gráfico seleccionados
def generate_output(change):
    global pivot_table
    variable = variable_dropdown.value
    chart_type = chart_type_dropdown.value
    products_selected = product_dropdown.value
    municipios_selected = municipio_dropdown.value
    index_field = index_dropdown.value
    x_field = x_axis_dropdown.value
    y_field = y_axis_dropdown.value
    
    # Filtrar el DataFrame por los productos seleccionados
    filtered_data = data[data["Producto"].isin(products_selected)]
    
    # Crear la tabla de pivote
    pivot_table = pd.pivot_table(filtered_data, values=variable, index=index_field, columns=["Producto", "Municipio"])
    
    
    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()
    
    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(pivot_table)

               
    # Generar y mostrar el gráfico
    with chart_output:
        try:
            fig = go.Figure()
            
            if chart_type == "bar":
                for product in products_selected:
                    for municipio in municipios_selected:
                        if product in pivot_table.columns.get_level_values("Producto") and municipio in pivot_table.columns.get_level_values("Municipio"):
                            fig.add_trace(go.Bar(
                                x=pivot_table.index,
                                y=pivot_table[(product, municipio)],
                                name=f"{product} ({municipio})"
                            ))
            else:
                for product in products_selected:
                    for municipio in municipios_selected:
                        if product in pivot_table.columns.get_level_values("Producto") and municipio in pivot_table.columns.get_level_values("Municipio"):
                            fig.add_trace(go.Scatter(
                                x=pivot_table[x_field][(product, municipio)],
                                y=pivot_table[y_field][(product, municipio)],
                                mode="lines+markers",
                                name=f"{product} ({municipio})"
                            ))

            fig.update_layout(
                title=f"Gráfico de {chart_type} de {variable} por {x_field} y {y_field}",
                xaxis=dict(title=x_field),
                yaxis=dict(title=y_field)
            )

            fig.show()
        except ValueError as e:
            print(f"No se pudo generar la gráfica. Error: {str(e)}")

            
# Asignar la función de generación de tabla y gráfico al evento "change" de los dropdown menús
variable_dropdown.observe(generate_output, 'value')
product_dropdown.observe(generate_output, 'value')
municipio_dropdown.observe(generate_output, 'value')
index_dropdown.observe(generate_output, 'value')
x_axis_dropdown.observe(generate_output, 'value')
y_axis_dropdown.observe(generate_output, 'value')
chart_type_dropdown.observe(generate_output, 'value')


# Función para exportar la tabla como archivo Markdown
def export_table_to_markdown(button):
    if pivot_table is None:
        print("No se ha generado ninguna tabla todavía.")
        return

    # Convertir la tabla a Markdown y guardarla en un archivo
    table = pivot_table.to_markdown()
    with open("tabla.md", "w") as f:
        f.write(table)
    
    print("La tabla se ha exportado exitosamente como archivo Markdown.")



# Botón para exportar la tabla
export_button = widgets.Button(description="Exportar tabla")
export_button.on_click(export_table_to_markdown)    


# Mostrar los dropdown menús, el botón y la salida de la tabla
display(variable_dropdown, product_dropdown, municipio_dropdown, index_dropdown, x_axis_dropdown, y_axis_dropdown, chart_type_dropdown)
display(export_button)
display(table_output)
display(chart_output)